In [1]:
import shapefile
import utm

In [16]:
def get_zone_conversion(latitude):
    ''' helper function for convert_to_utm. Given a latitude value, it 
        returns an approximation for how the utm coordinates change as
        the longitude crosses from UTM 17 to UTM 18.
        Adjust londelta to get a more precise answer'''
    londelta = .0000000001
    lat = latitude
    return utm.from_latlon(lat,-78 - londelta)[0] - utm.from_latlon(lat,-78 + londelta)[0]

def latlon_to_utm(lat, lon):
    new_coords = utm.from_latlon(lat,lon)
        
    # change the x coordinates if we are in zone 17
    if new_coords[2] == 17:
        return (new_coords[0] - get_zone_conversion(lat), new_coords[1])
    # otherwise leave it alone
    else:
        return tuple(new_coords[:2])
        
def convert_to_utm(lonlat_coords):
    ''' Convert a list of (lon, lat) coordinates to UTM. Assumes the order is 
        longitude, then latitude. This has been customized to work for New 
        York, so the input has to be in UTM zone 17 or 18. If it is
        in 17, the coordinates are converted to be in terms of zone 18'''
    utm_coords = []
    for (lon, lat) in lonlat_coords:
        utm_coords.append(latlon_to_utm(lat, lon))
    return utm_coords

In [23]:
configuration_filename = "config.txt"
origin_latlon = (41.99883715111737, -79.76162720445977)
origin_utm_18 = latlon_to_utm(origin_latlon[0], origin_latlon[1])
shapefile_name = "../Shapefiles/SouthernOnondaga/Shape/roads"
tile_size = 10000 # meters

In [25]:
def latlon_to_tile_id(lat, lon):
    (x, y) = latlon_to_utm(lat, lon)
    x -= origin_utm_18[0]
    y -= origin_utm_18[1]
    i = x // tile_size
    j = y // tile_size
    return (i, j)

In [28]:
print(latlon_to_tile_id(44.98726227282377, -73.37494146241328))

(52.0, 33.0)


In [29]:
def get_tile_bounds(i, j):
    min_x = i * tile_size
    max_x = (i + 1) * tile_size
    min_y = j * tile_size
    max_y = (j + 1) * tile_size
    return ((min_x, min_y), (max_x, max_y))

def get_slope(x1, y1, x2, y2):
    if x1 == x2:
        raise ValueError("Trying to compute slope of vertical segment %f %f %f %f" % (x1, y1, x2, y2))
    else:
        return (y2 - y1) / (x2 - x1)
    
def get_y_int(x1, y1, slope):
    return y1 - (x1 * slope)

def get_line_equation(x1, y1, x2, y2):
    slope = get_slope(x1, y1, x2, y2)
    y_int = get_y_int(x1, y1, slope)
    return (slope, y_int)

def cut_to_tile(i, j, x1, y1, x2, y2):
    """
    Return the segment resulting from intersecting the boundaries of tile i, j with
    the segment defined by (x1, y1), (x2, y2). Return [] if there is no intersection.
    """
    # These are the tile bounds
    ((min_x, min_y), (max_x, max_y)) = get_tile_bounds(i, j)
    
    # Check for no intersection
    if (y1 > max_y and y2 > max_y) or (y1 < min_y and y2 < min_y) or (x1 < min_x and x2 < min_x) or (x1 > max_x and x2 > max_x):
        return []
    # Moving forward, we know the segment does intersect the tile
    
    # Do the vertical line case first
    if x1 == x2:
        road_min = min(y1, y2)
        road_max = max(y1, y2)
        cut_min = max(road_min, min_y)
        cut_max = min(road_max, max_y)
        return ( (x1, cut_min), (x1, cut_max) )
    # Horizontal line is easy, too
    elif y1 == y2:
        road_min = min(x1, x2)
        road_max = max(x1, x2)
        cut_min = max(road_min, min_x)
        cut_max = min(road_max, max_x)
        return ( (cut_min, y1), (cut_max, y2) )
    # Diagonal line: more fun
    else:
        cut_segment = []
        slope, y_int = get_line_equation(x1, y1, x2, y2)
        
        # Check the south boundary
        x_of_intersection = (min_y - y_int) / slope
        if x_of_intersection >= min_x and x_of_intersection <= max_x:
            cut_segment.append( (x_of_intersection, min_y) )
        
        # Check the north boundary
        x_of_intersection = (max_y - y_int) / slope
        if x_of_intersection >= min_x and x_of_intersection <= max_x:
            cut_segment.append( (x_of_intersection, max_y) )
        
        # Check the west boundary
        y_of_intersection = slope * min_x + y_int
        if y_of_intersection >= min_y and y_of_intersection <= max_y:
            cut_segment.append( (min_x, y_of_intersection) )
        
        # Check the east boundary
        y_of_intersection = slope * max_x + y_int
        if y_of_intersection >= min_y and y_of_intersection <= max_y:
            cut_segment.append( (max_x, y_of_intersection) )
        
        # Check for interior points
        if (x1 > min_x and x1 < max_x) and (y1 > min_y and y1 < max_y):
            cut_segment.append( (x1, y1) )
        if (x2 > min_x and x2 < max_x) and (y2 > min_y and y2 < max_y):
            cut_segment.append( (x2, y2) )
        
        # Finally, check if something went wrong
        if len(cut_segment) != 2:
            raise Error("cut_segment contains %d points instead of 2" % len(cut_segment))
        
        return cut_segment